# Exploration
the goal of this notebook is to explore the data that we are working with

In [24]:
import pandas as pd
import os
import re
import plotly.express as px
from unidecode import unidecode
import json

---
# Concatenating Traning Files

In [25]:
def merge_training_files(training_folder, training_agr_path, verbose = False):
    # importing the csv files
    # The path for listing items
    path = training_folder

    # All files in the source folder
    files = os.listdir(path)

    # Filter out non training data
    r = re.compile("train-*")
    files = list(filter(r.match, files))

    # Aggregate training files
    valid_columns = ["tconst", "primaryTitle", "originalTitle", "startYear", "endYear", "runtimeMinutes", "numVotes", "label"]
    data = pd.DataFrame(columns=valid_columns)
    size_counter = 0
    for filename in files:
        filepath = path+filename
        _data = pd.read_csv(path+filename, index_col=0)
        data = pd.concat([data, _data], ignore_index=True)

        size_counter += _data.shape[0]
        if verbose: print(f'file: {filename} - {_data.shape[0]} | total {size_counter}')
    if verbose: print(f' >> data shape: {data.shape}')

    data.to_csv(training_agr_path)

In [26]:
# training_folder = './source_imdb/'
# data = merge_training_files(training_folder)

# Creating directors and writers files

In [ ]:
def construct_writers_directors(writer_path, director_path, output_path):
    with open(writer_path, 'r') as w:
        _ = json.load(w)
        writers = {}
        for dict in _:
            writers[dict["movie"]] = dict["writer"]

    with open(director_path, 'r') as d:
        _ = json.load(d)
        directors = {}
        for key, movie in _['movie'].items():
            directors[movie] = _["director"][key]

    # print(f'writer count: {len(writers)}, director count: {len(directors)}')

    d = {k: {'writer': w, 'director': directors[k]} for k,w in writers.items()}

    del _, writers, directors

    with open(output_path, "w") as f:
        json.dump(d, f)

In [28]:
# writer_path   = 'source_imdb/writing.json'
# director_path = 'source_imdb/directing.json'
# writer_director_path   = 'output/writers_directors.json'
#
# construct_writers_directors(writer_path, director_path, writer_director_path)

# Matching writers and directors to movies

In [ ]:
def writer_director_lookup(tconst, writer_director_dict):
    '''matches given tconst to writer and directs, pergorms a similarity check'''
    writer = writer_director_dict[tconst]['writer']
    director = writer_director_dict[tconst]['director']
    same = (writer == director)
    return writer, director, same

def match_writer_director(dataframe, writer_director_path):
    with open(writer_director_path, 'r') as w_d_json:
        w_d_dict = json.load(w_d_json)
        dataframe['writer'], dataframe['director'], dataframe['same_writer_director'] = \
            zip(*[writer_director_lookup(t, w_d_dict) for t in dataframe['tconst']])
    return dataframe

In [30]:
# data = match_writer_director(data, writer_director_path)
# data.head()

# fixing d-types

In [31]:
def fix_dtypes(dataframe):
    integer_columns  = ["startYear", "endYear", "runtimeMinutes", "numVotes"]
    bool_columns  = ["label", "same_writer_director"]
    string_columns  = ["tconst", "primaryTitle", "originalTitle", "writer", "director"]

    for c in integer_columns:
        dataframe[c] = pd.to_numeric(dataframe[c], errors="coerce")
    for c in string_columns:
        dataframe[c] = dataframe[c].astype('string')
    for c in bool_columns:
        if c in dataframe.columns:
            dataframe[c] = dataframe[c].astype('boolean')

    # print(dataframe.dtypes)

    return dataframe

# Cleaning text

In [98]:
# fix lexical bullshit from title column
def clean_text(text: str):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = unidecode(text)
    return text

---
# Brining the processing together

In [33]:
def clean_data(raw_data_path):
    """Join all the data preprocessing steps here"""
    dataframe = pd.read_csv(raw_data_path, index_col=0)
    dataframe = match_writer_director(dataframe, writer_director_path)
    dataframe = fix_dtypes(dataframe)
    return dataframe

def feature_engineer_data(dataframe):
    """All feature engineering steps go here"""
    return dataframe

In [43]:
# prexisting data
training_folder         = './source_imdb/'
validation_path         = 'source_imdb/validation_hidden.csv'
test_path               = 'source_imdb/test_hidden.csv'
writer_path             = 'source_imdb/writing.json'
director_path           = 'source_imdb/directing.json'

# our data
training_agr_path       = 'output/train_aggregated.csv'
writer_director_path    = 'output/writers_directors.json'

# construct information stores
merge_training_files(training_folder, training_agr_path, verbose = False)
construct_writers_directors(writer_path, director_path, writer_director_path)

# clean the three sets
train_clean = clean_data(training_agr_path)
valid_clean  = clean_data(validation_path)
test_clean  = clean_data(test_path)

# apply feature engineering steps
train_df = feature_engineer_data(train_clean)
valid_df  = feature_engineer_data(valid_clean)
test_df  = feature_engineer_data(test_clean)

---
# Model training

In [55]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch

In [36]:
h2o.init(max_mem_size_GB=10)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 9 mins
H2O_cluster_timezone:,Europe/Amsterdam
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_e_pg7917
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.87 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [51]:
# Set the predictors and response; set the response as a factor:
train_hf = h2o.H2OFrame(train_df)

y = "label"
X = train_hf.columns
X = list(filter(lambda x: x not in [y], X))
print(f'y: {y}')
print(f'X: {X}')

# Split the dataset into a train and valid set:
train, valid = train_hf.split_frame(ratios=[.8], seed=1234)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
y: label
X: ['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'numVotes', 'writer', 'director', 'same_writer_director']


In [58]:
xgb_params = {
        # 'learn_rate': [0.01, 0.1],
        'max_depth': [3, 5, 9],
        'sample_rate': [0.8, 1.0],
        'col_sample_rate': [0.2, 0.5, 1.0],
        # 'skip_drop': [0, 0.5, 1.0],
        'ntrees': [30, 50, 100, 200, 300],
}

# Train and validate a cartesian grid of GBMs
xgb_grid = H2OGridSearch(model=H2OXGBoostEstimator,
                                grid_id='xgb_grid_ntrees',
                                hyper_params=xgb_params)
xgb_grid.train(
    x=X, y=y,
    training_frame=train,
    validation_frame=valid,
    seed=35
)

# Get the grid results, sorted by validation RMSE
xgb_grid_ntrees_perf = xgb_grid.get_grid(sort_by='RMSE', decreasing=False)
xgb_grid_ntrees_perf

xgboost Grid Build progress: |█ (cancelled)
Errors/Warnings building gridsearch model

Hyper-parameter: col_sample_rate, 0.5
Hyper-parameter: learn_rate, 0.3
Hyper-parameter: max_depth, 3
Hyper-parameter: ntrees, 50
Hyper-parameter: sample_rate, 0.8
Hyper-parameter: skip_drop, 0.0
failure_details: Job Canceled
failure_stack_traces: java.lang.RuntimeException: Error while training XGBoost model
	at hex.tree.xgboost.XGBoost$XGBoostDriver.buildModelImpl(XGBoost.java:443)
	at hex.tree.xgboost.XGBoost$XGBoostDriver.buildModel(XGBoost.java:395)
	at hex.tree.xgboost.XGBoost$XGBoostDriver.computeImpl(XGBoost.java:381)
	at hex.ModelBuilder$Driver.compute2(ModelBuilder.java:247)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1658)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWork

H2OJobCancelled: Job<$03017f00000132d4ffffffff$_9f5a01bd9f5b7032af24a62992a14202> was cancelled by the user.

In [61]:
# Build and train the model:
imdb_xgb = H2OXGBoostEstimator(seed=10,
                               ntrees=200,
                               max_depth=6)
imdb_xgb.train(x=X,
               y=y,
               training_frame=train,
               validation_frame=valid)

xgboost Model Build progress: |

/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [primaryTitle, tconst]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1647970082100_4183


Model Summary: 


,,number_of_trees
0,,200.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.0871641245075922
RMSE: 0.29523571008194827
LogLoss: 0.29623792719762754
Mean Per-Class Error: 0.10271989103857493
AUC: 0.9631350157612215
AUCPR: 0.968267514213273
Gini: 0.926270031522443

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.44603567251137327: 


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-03-22 20:23:33,0.033 sec,0.0,0.500000,0.693147,0.500000,0.503604,1.000000,0.496396,0.500000,0.693147,0.500000,0.492074,1.000000,0.507926
1,,2022-03-22 20:23:33,0.128 sec,1.0,0.458871,0.613126,0.816924,0.842750,1.950728,0.271702,0.461035,0.617182,0.801606,0.803710,1.874155,0.278377
2,,2022-03-22 20:23:33,0.161 sec,2.0,0.437072,0.570702,0.824606,0.851408,1.985688,0.266688,0.441028,0.578069,0.808967,0.815938,1.964476,0.250476
3,,2022-03-22 20:23:33,0.197 sec,3.0,0.423267,0.542367,0.831758,0.857694,1.985688,0.261987,0.428343,0.552006,0.815697,0.823440,1.964476,0.269499
4,,2022-03-22 20:23:33,0.236 sec,4.0,0.414175,0.522453,0.835454,0.862137,1.985688,0.241147,0.420257,0.534384,0.819917,0.827634,1.974153,0.251110
5,,2022-03-22 20:23:33,0.284 sec,5.0,0.408541,0.509155,0.838765,0.866357,1.985688,0.254309,0.415518,0.523574,0.822105,0.835380,2.032216,0.248573
6,,2022-03-22 20:23:33,0.332 sec,6.0,0.403433,0.497334,0.843419,0.870357,1.985688,0.244751,0.412782,0.517076,0.823215,0.835405,2.032216,0.246037
7,,2022-03-22 20:23:33,0.382 sec,7.0,0.400397,0.489632,0.845793,0.873069,1.985688,0.240990,0.410858,0.512454,0.825168,0.837589,2.032216,0.250476
8,,2022-03-22 20:23:33,0.430 sec,8.0,0.397654,0.482720,0.848743,0.875827,1.985688,0.243184,0.409683,0.509238,0.826579,0.839560,2.032216,0.251110
9,,2022-03-22 20:23:33,0.480 sec,9.0,0.395230,0.476712,0.851408,0.878321,1.985688,0.243341,0.408832,0.507049,0.827322,0.838909,2.032216,0.255549



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,runtimeMinutes,3040.630371,1.000000,0.314347
1,numVotes,2943.498047,0.968055,0.304305
2,startYear,2407.962646,0.791929,0.248940
3,writer.\N,474.970215,0.156208,0.049103
4,same_writer_director.False,269.822388,0.088739,0.027895
5,originalTitle.<NA>,157.572235,0.051822,0.016290
6,endYear.2017.0,28.185413,0.009270,0.002914
7,endYear.2010.0,19.563330,0.006434,0.002022
8,writer.nm1347153,19.013681,0.006253,0.001966
9,endYear.2019.0,18.436451,0.006063,0.001906



See the whole table with table.as_data_frame()


In [62]:
# Eval performance:
perf = imdb_xgb.model_performance()

# Generate predictions on a test set (if necessary):
pred = imdb_xgb.predict(valid)

# Extract feature interactions:
feature_interactions = imdb_xgb.feature_interaction()

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


---
# Make predictions

In [65]:
valid_hf = h2o.H2OFrame(valid_df)
test_hf = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [66]:
pred_valid = imdb_xgb.predict(valid_hf)
pred_test = imdb_xgb.predict(test_hf)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |

/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'writer' has levels not trained on: ["nm0000045", "nm0000189", "nm0000219", "nm0000389", "nm0000485", "nm0000905", "nm0001328", "nm0001355", "nm0001357", "nm0001640", ...642 not listed..., "nm8276452", "nm8442541", "nm8664327", "nm8733312", "nm9212368", "nm9282965", "nm9572719", "nm9648068", "nm9803391", "nm9912021"]
  warnings.warn(w)
/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'director' has levels not trained on: ["nm0000045", "nm0000104", "nm0000578", "nm0000698", "nm0000797", "nm0001408", "nm0001471", "nm0001490", "nm0001692", "nm0001774", ...470 not listed..., "nm7920254", "nm7958316", "nm7978876", "nm8011325", "nm8381668", "nm8399396", "nm8734870", "nm9214307", "nm9290266", "nm9511013"]
  warnings.warn(w)
/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset

███████████████████████████████████████████████████| (done) 100%


/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'writer' has levels not trained on: ["nm0000033", "nm0000265", "nm0000485", "nm0000532", "nm0000578", "nm0000671", "nm0000698", "nm0000713", "nm0001024", "nm0001220", ...737 not listed..., "nm8369212", "nm8472096", "nm8704877", "nm8962876", "nm9135703", "nm9413392", "nm9448203", "nm9489621", "nm9832450", "nm9868437"]
  warnings.warn(w)
/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'director' has levels not trained on: ["nm0000059", "nm0000602", "nm0000671", "nm0000698", "nm0000797", "nm0001165", "nm0001428", "nm0001441", "nm0001554", "nm0001692", ...539 not listed..., "nm7121260", "nm7253772", "nm7263882", "nm7274555", "nm7278299", "nm8684196", "nm9135703", "nm9218697", "nm9522431", "nm9615170"]
  warnings.warn(w)
/Users/e/opt/anaconda3/lib/python3.9/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset

In [79]:
pred_test_list = h2o.as_list(pred_test)['predict'].tolist()
pred_test_list

with open('output/prediction_test.txt', 'w') as f:
    for item in pred_test_list:
        f.write("%s\n" % item)

pred_valid_list = h2o.as_list(pred_valid)['predict'].tolist()
pred_valid_list

with open('output/prediction_validation.txt', 'w') as f:
    for item in pred_valid_list:
        f.write("%s\n" % item)

In [96]:
# check F/F+T
print(f' test {pred_test_list.count(False)/(pred_test_list.count(False) + pred_test_list.count(True))}')

 test 0.427255985267035


In [97]:
# check F/F+T
print(f' valid {pred_test_list.count(False)/(pred_valid_list.count(False) + pred_valid_list.count(True))}')

 valid 0.48586387434554973
